# Exploratory Data Analysis
In the last class, we saw all the different ways in which hypothesis tests can fail. In reality, hypothesis testing is useful intellectual rigor in narrow circumstances: you have a pre-defined hypothesis, you have control (or understanding of how data are collected), and you observe everything relevant to your hypothesis. We will call this type of data analysis "confirmatory data analysis". It's kind of the last step of a data science workflow. 

What if you don't know what questions to ask? We contrast this process as "exploratory data analysis". Exploratory data analysis (EDA) is an approach to analyzing data sets to summarize their main characteristics, often with visual methods. A statistical model can be used or not, but primarily EDA is for seeing what the data can tell us beyond the formal modeling or hypothesis testing task. Exploratory data analysis includes the following steps:

* Specifying assumptions and detecting data errors
* Identifying relations between variables of interest
* Formulating a testable hypothesis (and a new experiment!!)

## Assumptions and Errors
The first step is to understand what we can assume about the dataset that we have. Some of the assumptions are extrinsic:
* How was the data collected?
* What was the data collection process trying to achieve? 
* How might the data collection process be biased?

Some of the assumptions are intrinsic:
* Data types of columns
* Relationships between columns
* More ???

In [82]:
import pandas as pd
df = pd.read_csv('ssn.csv', delimiter=' ')

In [83]:
df

,Name,Gender,Social,Age,Acct
0,Arthur-Crawford,male,092-30-2294,47,3882717958320846
1,Anthony-Dove,male,729-04-3185,54,7141086558511739
2,Noemi-Teel,female,117-67-4531,59,13061500012440717
3,Kim-Lovell,female,551-01-1394,51,1716351605690022
4,Kenneth-Gallagher,male,322-79-2084,36,13699495317963956
...,...,...,...,...,...
95,Ivory-Curry,male,275-17-1922,47,19231534736287234
96,Victor-Armstrong,male,846-57-2998,53,10498060972555052
97,Annie-Mayweather,female,693-63-9152,38,5563786638578513
98,Tracy-Zampieri,female,918-30-2569,44,1142720749472163


What kind of constraints do we have on this data? First, we may know that age has to be a positive number between 0 and 122.

In [84]:
df[df['Age'] < 0]

,Name,Gender,Social,Age,Acct
24,Robert-Prater,male,672-85-4374,-1,6251379381664522


We also know that social security numbers must be unique. How do we find uniqueness errors?

In [59]:
df.groupby('Social')['Social'].count()

Social
006-18-9627    1
008-99-9285    1
013-28-7669    1
020-89-8052    1
021-16-5299    1
              ..
922-22-4679    1
938-00-4773    1
950-04-7562    1
963-07-6350    1
966-26-8444    1
Name: Social, Length: 99, dtype: int64

In [60]:
social_count = df.groupby('Social')['Social'].count()
social_count[social_count > 1]

Social
092-30-2294    2
Name: Social, dtype: int64

In [61]:
social_count[social_count > 1].index

Index(['092-30-2294'], dtype='object', name='Social')

In [62]:
social_count[social_count > 1].index

Index(['092-30-2294'], dtype='object', name='Social')

In [67]:
for v in social_count[social_count > 1].index:
    print(v)

092-30-2294


In [68]:
for v in social_count[social_count > 1].index:
    print(v)
    print(df[df['Social'] == v])

092-30-2294
               Name Gender       Social  Age               Acct
0   Arthur-Crawford   male  092-30-2294   47   3882717958320846
19    Curtis-Martin   male  092-30-2294   50  10957389913694997


This seems like a lot of code to identify something pretty straight-forward. Is there a better way?

## Pandas Merge

Before, we dig deeper into error detection. Let's spend some time understanding how the merge function works in pandas. Pandas merge is equivalent to an "equality join" operation for those of you have taken a database course. Let's think of a simple example:

In [69]:
table1 = [{'name': 'John Doe', 'category': 'A'}, 
          {'name': 'Jane Smith', 'category': 'B'}, 
          {'name': 'Alex Taylor', 'category': 'A'},
          {'name': 'Brett Daniels', 'category': 'C'}]

table1_df = pd.DataFrame(table1)
table1_df

,name,category
0,John Doe,A
1,Jane Smith,B
2,Alex Taylor,A
3,Brett Daniels,C


In [70]:
table2 = [{'salary': 1000, 'category': 'A'}, 
          {'salary': 900, 'category': 'B'}, 
          {'salary': 500, 'category': 'C'}]
table2_df = pd.DataFrame(table2)
table2_df

,salary,category
0,1000,A
1,900,B
2,500,C


In [71]:
table1_df.merge(table2_df, on='category')

,name,category,salary
0,John Doe,A,1000
1,Alex Taylor,A,1000
2,Jane Smith,B,900
3,Brett Daniels,C,500


This function is commutative:

In [72]:
table2_df.merge(table1_df, on='category')

,salary,category,name
0,1000,A,John Doe
1,1000,A,Alex Taylor
2,900,B,Jane Smith
3,500,C,Brett Daniels


This function merges the two tables together on the category column and automatically removes the redundancy (1 single column is left). The behavior of this function can be subtle. Suppose, we change the category field to D for one of the rows:

In [73]:
table1 = [{'name': 'John Doe', 'category': 'A'}, 
          {'name': 'Jane Smith', 'category': 'B'}, 
          {'name': 'Alex Taylor', 'category': 'D'},
          {'name': 'Brett Daniels', 'category': 'C'}]

table1_df = pd.DataFrame(table1)
table1_df

,name,category
0,John Doe,A
1,Jane Smith,B
2,Alex Taylor,D
3,Brett Daniels,C


In [74]:
table1_df.merge(table2_df, on='category')

,name,category,salary
0,John Doe,A,1000
1,Jane Smith,B,900
2,Brett Daniels,C,500


The row gets dropped from the result! In the basic operating mode of the merge command any row that doesn't have a match gets dropped. There is a key word `how` that can modify this behavior. Suppose, we want the left rows that don't match:

In [75]:
table1_df.merge(table2_df, on='category', how='left')

,name,category,salary
0,John Doe,A,1000.0
1,Jane Smith,B,900.0
2,Alex Taylor,D,NaN
3,Brett Daniels,C,500.0


It returns those rows but with any additional columns null or nan, depending on the data type. If you set how to right you'll get the same answer as before (why?)

In [76]:
table1_df.merge(table2_df, on='category', how='right')

,name,category,salary
0,John Doe,A,1000
1,Jane Smith,B,900
2,Brett Daniels,C,500


## Checking Complex Conditions

How would you use a Pandas merge to generate an all pairs dataset?

In [85]:
def all_pairs(df):
    new_df = df.copy() # make a copy of the data frame
    new_df['dummy'] = 1
    
    return new_df.merge(new_df, on='dummy')

all_pairs(table1_df)

,name_x,category_x,dummy,name_y,category_y
0,John Doe,A,1,John Doe,A
1,John Doe,A,1,Jane Smith,B
2,John Doe,A,1,Alex Taylor,D
3,John Doe,A,1,Brett Daniels,C
4,Jane Smith,B,1,John Doe,A
5,Jane Smith,B,1,Jane Smith,B
6,Jane Smith,B,1,Alex Taylor,D
7,Jane Smith,B,1,Brett Daniels,C
8,Alex Taylor,D,1,John Doe,A
9,Alex Taylor,D,1,Jane Smith,B


Let's go back to our original dataset

In [78]:
pair_df = all_pairs(df)
pair_df

,Name_x,Gender_x,Social_x,Age_x,Acct_x,dummy,Name_y,Gender_y,Social_y,Age_y,Acct_y
0,Arthur-Crawford,male,092-30-2294,47,3882717958320846,1,Arthur-Crawford,male,092-30-2294,47,3882717958320846
1,Arthur-Crawford,male,092-30-2294,47,3882717958320846,1,Anthony-Dove,male,729-04-3185,54,7141086558511739
2,Arthur-Crawford,male,092-30-2294,47,3882717958320846,1,Noemi-Teel,female,117-67-4531,59,13061500012440717
3,Arthur-Crawford,male,092-30-2294,47,3882717958320846,1,Kim-Lovell,female,551-01-1394,51,1716351605690022
4,Arthur-Crawford,male,092-30-2294,47,3882717958320846,1,Kenneth-Gallagher,male,322-79-2084,36,13699495317963956
...,...,...,...,...,...,...,...,...,...,...,...
9995,Jane-Williamson,female,432-07-4044,24,488972166555225,1,Ivory-Curry,male,275-17-1922,47,19231534736287234
9996,Jane-Williamson,female,432-07-4044,24,488972166555225,1,Victor-Armstrong,male,846-57-2998,53,10498060972555052
9997,Jane-Williamson,female,432-07-4044,24,488972166555225,1,Annie-Mayweather,female,693-63-9152,38,5563786638578513
9998,Jane-Williamson,female,432-07-4044,24,488972166555225,1,Tracy-Zampieri,female,918-30-2569,44,1142720749472163


In [79]:
pair_df[(pair_df['Social_x'] == pair_df['Social_y']) & (pair_df['Acct_x'] != pair_df['Acct_y'])]

,Name_x,Gender_x,Social_x,Age_x,Acct_x,dummy,Name_y,Gender_y,Social_y,Age_y,Acct_y
19,Arthur-Crawford,male,092-30-2294,47,3882717958320846,1,Curtis-Martin,male,092-30-2294,50,10957389913694997
1900,Curtis-Martin,male,092-30-2294,50,10957389913694997,1,Arthur-Crawford,male,092-30-2294,47,3882717958320846


Why did we use the single (bitwise) and? It's because the python `and` is abiguous for an entire boolean mask. This works for more complicated examples as well:

In [80]:
emps = [{'name': 'John Doe', 'rank': 'Manager', 'salary': 100}, 
          {'name': 'Jane Smith', 'rank': 'Manager', 'salary': 55}, 
          {'name': 'Alex Taylor', 'rank': 'Employee', 'salary': 32},
          {'name': 'Brett Daniels', 'rank': 'Employee', 'salary': 57}]
emps_df = pd.DataFrame(emps)
emps_df

,name,rank,salary
0,John Doe,Manager,100
1,Jane Smith,Manager,55
2,Alex Taylor,Employee,32
3,Brett Daniels,Employee,57


Suppose, we have the integrity constraint: no manager can earn less than an employee.

In [87]:
pair_df = all_pairs(emps_df)
pair_df[(pair_df['rank_x'] == 'Manager') &(pair_df['rank_y'] == 'Employee') & (pair_df['salary_x'] < pair_df['salary_y'])]

,name_x,rank_x,salary_x,dummy,name_y,rank_y,salary_y
7,Jane Smith,Manager,55,1,Brett Daniels,Employee,57
